In [1]:
#DNA is formed by two-links chains of four nitrogen bases: adenine (A), cytosine (C), guanine (G) and thymine(T)
#Having an even number of links, a DNA chain is formed by the combinations A-T and C-G

#Exploring the whole space of combinations generated by the nitrogen bases on the number of links, we want to 
#find the specific ones that form a DNA chain

#This task can be performed by means of the Grover's search algorithm, being quadratically more efficient than
#classical search algorithms


import sys
import numpy as np
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, transpile
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.tools.visualization import circuit_drawer


In [23]:
## ISSUES: What is the optimal number of Grover iterations? Why increasing them makes it unstable?
## The circuit design is not optimal. For the case of nchain=1 and n_grover=1, the outputs are the four possible
## solutions only, as we should expect. This is related to how you encode the solutions of a chain to the others
## in ancilla and onto the output qubit. Check this as well. 

n_grover = 4 #Grover search iterations


nchains = 1 #Number of DNA chains


links = 2*nchains         #Number of DNA links
nqubits = 2*links
repeat = round(nqubits/4) #How many times we will repeat the conditions every four qubits


# DNA and qubit maps
dna_map   = [ 'A',  'G',  'C',  'T']
qubit_map = ['00', '01', '10', '11']


# Start the circuit
inputs = QuantumRegister(nqubits, name='x')
conditions = QuantumRegister(nqubits, name='c')
conditions_ancilla = QuantumRegister(nchains, name='c_A')
output = QuantumRegister(1, name='out')
measure = ClassicalRegister(nqubits, name='measure')

qc = QuantumCircuit(inputs, conditions, conditions_ancilla, output,measure)

# Output qubit in state |->
qc.h(output)
qc.z(output)

# Initialize qubits in state |s>
qc.h(inputs)

qc.barrier()



#Start Grover iterations
for ng in range(n_grover):

 c = 0
 for r in range(repeat):
  c1, c2 = c, c+3
  #We set now two different sets of conditions
  cq1 = conditions[c1:c2] 
  cq2 = [conditions[c1],conditions[c1+1],conditions[c1+3]]
    
  #First condition
  qc.cx(inputs[c1  ], conditions[c1])
  qc.cx(inputs[c1+2], conditions[c1])

  #Second condition
  qc.cx(inputs[c1+1], conditions[c1+1])
  qc.cx(inputs[c1+3], conditions[c1+1])

  #Third condition
  qc.cx(inputs[c1+1], conditions[c1+2])
  qc.cx(inputs[c1+2], conditions[c1+2])

  #Fourth condition
  qc.cx(inputs[c1  ], conditions[c1+3])
  qc.cx(inputs[c1+1], conditions[c1+3])
    
  #Meet conditions in output qubit
  #These ancilla is what makes the code to bug when going beyond one DNA chain...
  #qc.mct(cq1, conditions_ancilla[r])
  #qc.mct(cq2, conditions_ancilla[r])

  qc.mct(cq1, output)
  qc.mct(cq2, output)

  c = c+4
 #This ancilla is what makes the code to bug when going beyond one DNA chain. Related to former comment
 #qc.mct(conditions_ancilla, output)

 qc.barrier() 

 #Put them back to zero
 c = 0
 for r in range(repeat):
  c1, c2 = c, c+3
  #We set now two different sets of conditions
  cq1 = conditions[c1:c2] 
  cq2 = [conditions[c1],conditions[c1+1],conditions[c1+3]]
    
  #First condition
  qc.cx(inputs[c1  ], conditions[c1])
  qc.cx(inputs[c1+2], conditions[c1])

  #Second condition
  qc.cx(inputs[c1+1], conditions[c1+1])
  qc.cx(inputs[c1+3], conditions[c1+1])

  #Third condition
  qc.cx(inputs[c1+1], conditions[c1+2])
  qc.cx(inputs[c1+2], conditions[c1+2])

  #Fourth condition
  qc.cx(inputs[c1  ], conditions[c1+3])
  qc.cx(inputs[c1+1], conditions[c1+3])

  qc.barrier() 
    

  c = c+4
    
 #Phase amplification
 qc.h(inputs)
 qc.x(inputs)

 qc.h(0)
 qc.mcx(list(range(1,nqubits)), 0)
 qc.h(0)

 qc.x(inputs)
 qc.h(inputs)

qc.barrier() 

# Measurement
qc.measure(inputs, measure)


qc.draw(fold=-1)
# To make a pretty plot of the circuit 
#circuit_drawer(qc, output='mpl', style={'backgroundcolor': '#EEEEEE'})
# To export the circuit to a Latex file 
#qc.draw('latex_source', filename='./filename.tex')



┌───┐      ░                                                    ░                                          ░ ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐                                              ░                                          ░ ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐                                              ░                                          ░ ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐                                              ░                                          ░ ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐ ░ ┌─┐         
      x_0: ┤ H ├──────░───■────────────────────────■───────────────────────░───■────────────────────────■─────────────░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├──■───────────────────■───────────────────────░───■────────────────────────■─────────────░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├──■───────────────────■───────────────────────░───■────────────────────────■─────────────░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├──■───────────────────■───────────────────────░───■────────────────────────■─────────────░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─░─┤M├─────────
           ├───┤      ░   │                        │                       ░   │                        │             ░ ├───┤├───┤└───┘└─┬─┘├───┤├───┤└───┘  │                   │                       ░   │                        │             ░ ├───┤├───┤└───┘└─┬─┘├───┤├───┤└───┘  │                   │                       ░   │                        │             ░ ├───┤├───┤└───┘└─┬─┘├───┤├───┤└───┘  │                   │                       ░   │                        │             ░ ├───┤├───┤└───┘└─┬─┘├───┤├───┤└───┘ ░ └╥┘┌─┐      
      x_1: ┤ H ├──────░───┼────■──────────────■────┼─────────■─────────────░───┼────■──────────────■────┼─────────■───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──■────┼─────────■─────────┼─────────■─────────────░───┼────■──────────────■────┼─────────■───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──■────┼─────────■─────────┼─────────■─────────────░───┼────■──────────────■────┼─────────■───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──■────┼─────────■─────────┼─────────■─────────────░───┼────■──────────────■────┼─────────■───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░──╫─┤M├──────
           ├───┤      ░   │    │              │    │         │             ░   │    │              │    │         │   ░ ├───┤├───┤       │  ├───┤├───┤  │    │         │         │         │             ░   │    │              │    │         │   ░ ├───┤├───┤       │  ├───┤├───┤  │    │         │         │         │             ░   │    │              │    │         │   ░ ├───┤├───┤       │  ├───┤├───┤  │    │         │         │         │             ░   │    │              │    │         │   ░ ├───┤├───┤       │  ├───┤├───┤      ░  ║ └╥┘┌─┐   
      x_2: ┤ H ├──────░───┼────┼────■─────────┼────┼────■────┼─────────────░───┼────┼────■─────────┼────┼────■────┼───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──┼────┼─────────┼────■────┼────■────┼─────────────░───┼────┼────■─────────┼────┼────■────┼───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──┼────┼─────────┼────■────┼────■────┼─────────────░───┼────┼────■─────────┼────┼────■────┼───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──┼────┼─────────┼────■────┼────■────┼─────────────░───┼────┼────■─────────┼────┼────■────┼───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────░──╫──╫─┤M├───
           ├───┤      ░   │    │    │         │    │    │    │             ░   │    │    │         │    │    │    │   ░ ├───┤├───┤       │  ├───┤├───┤  │    │         │    │    │    │    │             ░   │    │    │         │    │    │    │   ░ ├───┤├───┤       │  ├───┤├───┤  │    │         │    │    │    │    │             ░   │    │    │         │    │    │    │   ░ ├───┤├───┤       │  ├───┤├───┤  │    │         │    │    │    │    │             ░   │    │    │         │    │    │    │   ░ ├───┤├───┤       │  ├───┤├───┤      ░  ║  ║ └╥┘┌─┐
      x_3: ┤ H ├──────░───┼────┼────┼────■────┼────┼────┼────┼─────────────░───┼────┼────┼────■────┼────┼────┼────┼───░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──┼────┼────■────┼

In [21]:
# Simulation
aer_simulator = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_simulator)
result = aer_simulator.run(transpiled_qc).result()

# Plot probability distribution (not the prettiest plot)
plot_distribution(result.get_counts(), figsize=(50, 5), bar_labels=0, filename="dist_general.pdf")


results_dict = result.get_counts()

sorted_results = sorted(results_dict.items(), key=lambda x:x[1], reverse=True)

# Print the names of the columns.
print("{:<14} {:<14}".format('#', 'DNA CHAIN'))

# Get the first three solutions
cs = 1
for key, value in sorted_results[0:3]:
 key_str = [key[i:i+2] for i in range(0, nqubits, 2)]
 sol_str = ''
 for k in key_str:
  sol_str = sol_str + dna_map[qubit_map.index(k)]
 print("{:<14} {:<14}".format(cs, sol_str))
 cs += 1   


print(" ")
print("ALL POSSIBLE OUTCOMES")
print(" ")
# Print the names of the columns.
print("{:<14} {:<14}".format('SOLUTION', 'COUNTS'))



    
#All solutions
for key, value in sorted_results:
    print("{:<14} {:<14}".format(key, value))
    
    




#              DNA CHAIN     
1              AT            
2              TA            
3              CG            
 
ALL POSSIBLE OUTCOMES
 
SOLUTION       COUNTS        
0011           274           
1100           270           
1001           243           
0110           237           
